In [ ]:
# hier ist die testumgebung für zahlen
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from chronos import ChronosPipeline


In [ ]:

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-large",
  device_map="cpu",
  torch_dtype=torch.bfloat16,
)


In [ ]:

# we want a plot with percent in range for 
import random
# daten
import os, json
wd = "/home/a2/code/fin/tradingbot/data/stocks"
sxsx = os.listdir(wd)



In [1]:
a = "asjd"
a[2:]

'jd'

In [ ]:

with open("prediction.json", "r") as f:
    store = json.load(f) # content: {"name", "ctxt_len", "prediction"}
import plotly.graph_objects as go
prediction_length = 25
count_arr = [0 for _ in range(25)]
count_5 = 0
boolean_count_5d_return = 0
for s in sxsx:
    for p in store:
        if p["name"] == s:
            break
    else:
        continue  
    p = os.path.join(wd, s)
    df = pd.read_csv(p)
    size = len(df)
    t_main = torch.tensor(df["Close"]) # here we try to randomize days to minimize makro
    context = t_main[p["ctxt_len"]:] # correct name 

    forecast = p["prediction"]

    low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)
    r = t_main[-prediction_length:] # real data
    # perc of in 80% interval for 25 time steps
    b_in_interval = True
    # 5d acc
    last_val = context[-1] # referenzwert
    fifth_day_interval = (low[4], high[4])
    respective_returns = ((fifth_day_interval[0]-last_val)/last_val, (fifth_day_interval[1]-last_val)/last_val)
    real_return = (t_main[-21]-last_val)/last_val
    if real_return>respective_returns[0] and real_return<respective_returns[1]:
        count_5 += 1
    if (abs(respective_returns[0])-abs(respective_returns[1]) < 0):
        if real_return > 0:
            boolean_count_5d_return += 1
    else:
        if real_return<0:
            boolean_count_5d_return += 1

    for n, i in enumerate(r):
        if not (i > low[n] and i < high[n]):
            b_in_interval = False
            break
    if b_in_interval:
        count_arr[n] += 1
    
# maybe also look at 80 perc interval and try to put a 5 day acuracy
    # plot this shit 
my_length = len(sxsx)
print(f"after 5 days in interval: {count_5/my_length}")
print(f"after 5 days right estimate of up/down: {boolean_count_5d_return/my_length}")
count_arr = list(map(lambda x: x/my_length, count_arr))
fig = go.Figure()
fig.add_trace(go.Scatter(y=count_arr, mode='lines+markers', name='80 perc interval acc'))
fig.show()

# 0.12
# save data

with open("predictions.json", "w") as f:
    json.dump(store, f)


In [ ]:

with open("predictions.json", "w") as f:
    json.dump(store, f)
df = pd.read_csv(p)
len(df)
a = [0,0,0,0,0,0]
a[-0]
